EASY GUIDES AI Assistant-with Authentication


In [3]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [4]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4.1-mini"
openai = OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
# MODEL = "llama3.2"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


OpenAI API Key exists and begins sk-proj-


In [5]:
system_message = """
You are a helpful assistant for an Airline called EASYGUIDE.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. u can get the ticket prices based on what city user asked and before updating 
you need to authorize the user whether he/she has access to change the content"""

The Below Json is the description of what each function does cause GPT will understand the functionality of a particular function better by using JSON

In [6]:
# describe our functions:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

update_price_function = {
    "name": "update_ticket_price",
    "description": "update the price of a ticket if a record with the city exists else create a new record with the price and city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city that we need to add or update the ticket price",
            },
            "price": {
                "type": "string",
                "description": "The ticket price for the city",
            },
        },
        "required": ["city", "price"],
        "additionalProperties": False
    }
}

authentication_function = {
    "name": "authenticate_user",
    "description": """this function is used to authenticate the user before updating the ticket price so in order to update he should have admin role,
        this function will fetch the role of use by using his name and return true if the user has admin role or false""",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "name of the user"
            },
        },
        "required": ["name"],
        "additionalProperties": False
    }
}



ADD the above function description into the tools list

In [7]:
# And this is included in a list of tools:

tools = [
    {
        "type": "function",
        "function": price_function
    },
    {
        "type": "function",
        "function": update_price_function
    },
    {
        "type": "function",
        "function": authentication_function
    }
]

## using sql as a tool

we are going to fetch, update the price and authenticate the user by using sql DB

In [8]:
import sqlite3


In [9]:
DB = "EasyGuides.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS prices (city TEXT PRIMARY KEY, price REAL)')
    cursor.execute('CREATE TABLE IF NOT EXISTS roles (name TEXT PRIMARY KEY, role TEXT )')
    conn.commit()

In [10]:
def get_ticket_price(destination_city):
    print(f"DATABASE TOOL CALLED: Getting price for {destination_city}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city = ?', (destination_city.lower(),))
        result = cursor.fetchone()
        return f"Ticket price to {destination_city} is ${result[0]}" if result else "No price data available for this city"

Adding the Ticket Prices Records into the prices Table

In [11]:
def set_ticket_price(city, price):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (city, price) VALUES (?, ?) ON CONFLICT(city) DO UPDATE SET price = ?', (city.lower(), price, price))
        conn.commit()

In [12]:
ticket_prices = {"london":799, "paris": 899, "tokyo": 1420, "sydney": 2999}
for city, price in ticket_prices.items():
    set_ticket_price(city, price)

Adding the user roles into roles table 

In [13]:
def add_user_role(name, role):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO roles (name,role) VALUES (?, ?) ON CONFLICT(name) DO UPDATE SET role = ?', (name, role,role))
        conn.commit()

In [14]:
user_role_list = {"tarun":"admin", "sai": "user", "kiran": "user", "jeff":"admin"}
for name, role in user_role_list.items():
    add_user_role(name, role)

Below function will Authenticate the user

In [15]:
def authenticate_user(name):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT role FROM roles WHERE name = ?', (name,))
        result=cursor.fetchone()
        if not result:
            print(f"User '{name}' not found.")
            return "no user found with the given name"

        role = result[0].lower()
        print(f"User '{name}' has role: {role}")

        if role == "admin":
            return "user has admin role"
        else:
            return "user is not admin"


Function to Update the Ticket Price

In [16]:
def update_ticket_price(city, price):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute(
            'INSERT INTO prices (city, price) VALUES (?, ?) '
            'ON CONFLICT(city) DO UPDATE SET price = ?',
            (city.lower(), price, price)
        )
        conn.commit()
        return f"Ticket price to {city} is updated to {price}"

In [17]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    print(response)

    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    return response.choices[0].message.content

In [18]:
# Map function names to real implementations
TOOL_FUNCTIONS = {
    "get_ticket_price": get_ticket_price,
    "update_ticket_price": update_ticket_price,
    "authenticate_user": authenticate_user
}

def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        func_name = tool_call.function.name
        print(func_name)
        arguments = json.loads(tool_call.function.arguments)

        # Lookup the function dynamically
        func = TOOL_FUNCTIONS.get(func_name)
        if not func:
            responses.append({
                "role": "tool",
                "content": f"Unknown tool function: {func_name}",
                "tool_call_id": tool_call.id,
            })
            continue

        # Match argument names automatically using unpacking (**)
        try:
            result = func(**arguments)
        except Exception as e:
            result = f"Error calling {func_name}: {e}"
        print("result",result)
        responses.append({
            "role": "tool",
            "content": result,
            "tool_call_id": tool_call.id,
        })

    return responses


In [19]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7911
* To create a public link, set `share=True` in `launch()`.


ChatCompletion(id='chatcmpl-CV3TDHyZDbE4JpxFO6qLSub4ncMIM', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you with EASYGUIDE Airlines today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1761517543, model='gpt-4.1-mini-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_4c2851f862', usage=CompletionUsage(completion_tokens=15, prompt_tokens=258, total_tokens=273, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
ChatCompletion(id='chatcmpl-CV3TPXTQk2RwgvFxps4390qTm0lSo', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, fu